In [8]:
! pip install evaluate
!pip install rouge_score  
! pip install Bert_score

  Preparing metadata (setup.py) ... done
  Obtaining dependency information for absl-py from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 2.3 MB/s eta 0:00:00 0:00:01
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=6142bcbfd0953eb16f31ea79130ab05b24c1f13aa2ea1abbc9b941484f682eb5
  Stored in directory: /Users/abhignaragala/Library/Caches/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [9]:
import pandas as pd
from evaluate import load

# Load the ROUGE score model
rouge = load('rouge')

# Read data from CSV file
df = pd.read_csv("final.csv")

# Extract actual and predicted values from the DataFrame
predictions = df["Predicted"].tolist()
references = df["Actual"].tolist()

# Compute ROUGE scores for each pair of actual and predicted values
results = rouge.compute(predictions=predictions, references=references)

# Save the ROUGE scores to a new CSV file
output_df = pd.DataFrame({"Actual": references, "Predicted": predictions, "ROUGE-1": results['rouge1'], "ROUGE-2": results['rouge2'], "ROUGE-L": results['rougeL'], "ROUGE-Lsum": results['rougeLsum']})

output_df.to_csv("rouge_scores_output.csv", index=False)

# Print the ROUGE scores
print("ROUGE Scores:")
print("ROUGE-1:", results['rouge1'])
print("ROUGE-2:", results['rouge2'])
print("ROUGE-L:", results['rougeL'])
print("ROUGE-Lsum:", results['rougeLsum'])

ROUGE Scores:
ROUGE-1: 0.3680666846515993
ROUGE-2: 0.16418500665533092
ROUGE-L: 0.23349828360409808
ROUGE-Lsum: 0.2331992695246869


In [13]:
import pandas as pd
from evaluate import load

# Load the BERT score model
bertscore = load("bertscore")

# Read data from CSV file
df = pd.read_csv("final.csv")

# Extract actual and predicted values from the DataFrame
predictions = df["Predicted"].tolist()
references = df["Actual"].tolist()

# Compute BERT scores for each pair of actual and predicted values
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")

# Extract individual BERT scores (precision, recall, F1) for each pair
precision = results["precision"]
recall = results["recall"]
f1 = results["f1"]

# Create a new DataFrame to store the BERT scores
output_df = pd.DataFrame({"Actual": references, "Predicted": predictions, "Precision": precision, "Recall": recall, "F1": f1})

# Save the BERT scores to a new CSV file
output_df.to_csv("bert_scores_output.csv", index=False)

# Calculate the final cumulative BERT scores
final_precision = sum(precision) / len(precision)
final_recall = sum(recall) / len(recall)
final_f1 = sum(f1) / len(f1)

# Print the final cumulative BERT scores
print("Final Cumulative BERT Scores:")
print("Precision:", final_precision)
print("Recall:", final_recall)
print("F1 Score:", final_f1)

Final Cumulative BERT Scores:
Precision: 0.7835351845726894
Recall: 0.8427686490605225
F1 Score: 0.8117149169720597


In [12]:
import pandas as pd
from evaluate import load
from nltk.translate.bleu_score import sentence_bleu

# Load BLEU score function from NLTK
def compute_bleu_score(prediction, references):
    return sentence_bleu(references, prediction)

# Read data from CSV file
df = pd.read_csv("final.csv")

# Extract actual and predicted values from the DataFrame
predictions = df["Predicted"].tolist()
references_list = df["Actual"].apply(lambda x: [reference.strip() for reference in x.split(',')]).tolist()

# Compute BLEU scores for each pair of actual and predicted values
bleu_scores = [compute_bleu_score(prediction, references) for prediction, references in zip(predictions, references_list)]

# Create a new DataFrame to store the BLEU scores
output_df = pd.DataFrame({"Actual": df["Actual"], "Predicted": predictions, "BLEU Score": bleu_scores})

# Save the BLEU scores to a new CSV file
output_df.to_csv("bleu_scores_output.csv", index=False)

# Calculate the average BLEU score
average_bleu_score = sum(bleu_scores) / len(bleu_scores)

# Print the average BLEU score
print("Average BLEU Score:", average_bleu_score)

Average BLEU Score: 0.29347809689525894
